In [35]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [36]:
spark = SparkSession.builder.appName("temp").master("local[*]").getOrCreate()

In [37]:
df = spark.read.parquet("../data/result_1m")

In [38]:
df.count()

488244

In [48]:
predict_sum_df = df.groupBy("block_height").agg({"predict": "sum"}).withColumnRenamed("sum(predict)", "predict_sum")
dt = df.join(predict_sum_df, on="block_height", how="left")

In [42]:
dt.count()

488244

In [50]:
dt = dt.withColumn("percent", dt.predict / dt.predict_sum)

In [52]:
dt = dt.drop("predict_sum")

In [53]:
dt = dt.drop("__index_level_0__")

In [ ]:
dt.show(100, truncate=False)

In [55]:
dt.repartition(1).write.parquet("../data/percent.parquet")